# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [3]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [4]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,2,Butaritari,KI,81.43,light rain,3.0707,172.7902,No hotel found
1,5,Bairiki,KI,80.60,light intensity shower rain,1.3292,172.9752,Marry's Motel
2,7,Axim,GH,78.22,light rain,4.8699,-2.2405,Axim Beach Hotel
3,8,Hilo,US,73.26,overcast clouds,19.7297,-155.0900,Dolphin Bay Hotel
4,10,Muncar,ID,84.76,overcast clouds,-8.4333,114.3333,No hotel found


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [5]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Current Description"]
)


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [8]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City"]=="Veraval"]
vacation_end = vacation_df.loc[vacation_df["City"]=="Veraval"]
vacation_stop1 = vacation_df.loc[vacation_df["City"]=="Baleshwar"]
vacation_stop2 = vacation_df.loc[vacation_df["City"]=="Mamallapuram"]
vacation_stop3 = vacation_df.loc[vacation_df["City"]=="Vinukonda"]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [9]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start,vacation_stop1,vacation_stop2,vacation_stop3,vacation_end],ignore_index=True)

# Display sample data
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,97,Veraval,IN,72.81,clear sky,20.9000,70.3667,Shyam
1,550,Baleshwar,IN,73.99,broken clouds,21.4942,86.9317,Dropadi
2,413,Mamallapuram,IN,77.05,scattered clouds,12.6264,80.1722,Radisson Temple Bay
3,264,Vinukonda,IN,79.27,clear sky,16.0500,79.7500,BhagyaSree Loadge
4,97,Veraval,IN,72.81,clear sky,20.9000,70.3667,Shyam


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [10]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["Lat", "Lng"]].copy()

# Display sample data
waypoints_df.head()

,Lat,Lng
0,20.9000,70.3667
1,21.4942,86.9317
2,12.6264,80.1722
3,16.0500,79.7500
4,20.9000,70.3667


## Use GeoViews to create map that shows the four cities in the itinerary

In [15]:
# Configure the map plot by using the itineraty_df
waypoints_map = itinerary_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

In [16]:
# Display the route_map.
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [17]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [18]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': 'e402aa32d77a41f4b900ccfedf9adaeb',
 'waypoints': '20.9,70.3667|21.4942,86.9317|12.6264,80.1722|16.05,79.75|20.9,70.3667'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [19]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()

In [26]:
route_response

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [70.3667, 20.9], 'original_index': 0},
     {'location': [86.9317, 21.4942], 'original_index': 1},
     {'location': [80.1722, 12.6264], 'original_index': 2},
     {'location': [79.75, 16.05], 'original_index': 3},
     {'location': [70.3667, 20.9], 'original_index': 4}],
    'units': 'metric',
    'distance': 5983080,
    'distance_units': 'meters',
    'time': 248588.515,
    'legs': [{'distance': 2295656,
      'time': 95514.82,
      'steps': [{'from_index': 0,
        'to_index': 4,
        'distance': 37,
        'time': 5.432,
        'instruction': {'text': 'Drive west.'}},
       {'from_index': 4,
        'to_index': 10,
        'distance': 155,
        'time': 14.34,
        'instruction': {'text': 'Turn right.'}},
       {'from_index': 10,
        'to_index': 38,
        'distance': 661,
        'time': 43.065,
        'instruction': {'text': 'Turn left.'}},
       {'from_inde

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [24]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]["properties"]["legs"]

In [25]:
legs

[{'distance': 2295656,
  'time': 95514.82,
  'steps': [{'from_index': 0,
    'to_index': 4,
    'distance': 37,
    'time': 5.432,
    'instruction': {'text': 'Drive west.'}},
   {'from_index': 4,
    'to_index': 10,
    'distance': 155,
    'time': 14.34,
    'instruction': {'text': 'Turn right.'}},
   {'from_index': 10,
    'to_index': 38,
    'distance': 661,
    'time': 43.065,
    'instruction': {'text': 'Turn left.'}},
   {'from_index': 38,
    'to_index': 41,
    'distance': 135,
    'time': 15.656,
    'instruction': {'text': 'Turn right onto Rajmahal Road.'}},
   {'from_index': 41,
    'to_index': 113,
    'distance': 2517,
    'time': 160.728,
    'instruction': {'text': 'Turn left to stay on Rajmahal Road.'}},
   {'from_index': 113,
    'to_index': 170,
    'distance': 3270,
    'time': 157.481,
    'instruction': {'text': 'Turn left.'}},
   {'from_index': 170,
    'to_index': 261,
    'distance': 4249,
    'time': 178.246,
    'instruction': {'text': 'Turn right onto NH51.'

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [47]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for index in legs:
    lat = legs["steps"]["instruction"]
    lng = legs["steps"]["instruction"]

    
    latitude.append(lat)
    longitude.append(lng)

TypeError: list indices must be integers or slices, not str

In [38]:
longitude

[{'distance': 1488201,
  'time': 60982.363,
  'steps': [{'from_index': 0,
    'to_index': 12,
    'distance': 529,
    'time': 47.644,
    'instruction': {'text': 'Drive west.'}},
   {'from_index': 12,
    'to_index': 14,
    'distance': 120,
    'time': 9.217,
    'instruction': {'text': 'Turn right onto Fakir Mohan College Road.'}},
   {'from_index': 14,
    'to_index': 20,
    'distance': 685,
    'time': 49.406,
    'instruction': {'text': 'Bear left.'}},
   {'from_index': 20,
    'to_index': 63,
    'distance': 5220,
    'time': 318.717,
    'instruction': {'text': 'Turn left.'}},
   {'from_index': 63,
    'to_index': 564,
    'distance': 97088,
    'time': 3913.307,
    'instruction': {'text': 'Turn left onto NH16/AH45.'}},
   {'from_index': 564,
    'to_index': 1072,
    'distance': 68433,
    'time': 2745.449,
    'instruction': {'text': 'Keep right to stay on NH16.'}},
   {'from_index': 1072,
    'to_index': 1434,
    'distance': 20031,
    'time': 821.246,
    'instruction': 

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [14]:
# Create an empty DataFrame to store the steps' coordinates
route_df = # YOUR CODE HERE

# Add the steps' longitude and latitude from each step as columns to the DataFrame
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

,longitude,latitude
0,12.479944,41.889982
1,12.479927,41.890012
2,12.479760,41.890249
3,12.479580,41.890493
4,12.479418,41.890662


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [15]:
# Configure the route path by using the GeoViews' Path function
route_path = # YOUR CODE HERE

In [16]:
# Display a composed plot by using the route_map and route_path objects
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)
   .Path.I   :Path   [longitude,latitude]